In [1]:
from batching_algs import *
from train import *

In [2]:
import sys
sys.path.append("../")

from projects import project_list
from batch_updation import *

In [3]:
batch_total = 0
algorithm = ['BATCHBISECT', 'BATCHDIVIDE4', 'BATCHSTOP4']
confidence = list(range(2,21))

In [4]:
result_file = open('all_timeout_results.csv', 'w')
result_headers = ['version', 'project', 'algorithm', 'batch_size', 'confidence', 'project_reqd_builds', 'project_missed_builds', 'project_saved_builds', 'project_delays', 'testall_size', 'batch_delays', 'batch_median', 'ci']
writer = csv.writer(result_file)
writer.writerow(result_headers)
result_file.close()

In [5]:
def get_minimum_for(alg):
    if alg == 'BATCHBISECT':
        return 1
    return 4

In [6]:
def static_rule(p, ver):
    
    
    global batch_total
    global batch_durations
    
    p_name = p.split('/')[1]
    result_file = open('all_timeout_results.csv', 'a+')
    writer = csv.writer(result_file)

    
    X_test = sbs(p, ver, train=0)
    if len(X_test) == 0:
        return
    
    dir_name = p_name[:-4].split('-', 1)[1]
    model_file_name = '../../Models/' + dir_name + '/' + p_name + '_' + str(ver) + '_best_model.pkl'
    model_file = open(model_file_name, 'rb')
    predictor = pickle.load(model_file)
    threshold = pickle.load(model_file)
    
    Y_test = X_test['tr_status'].tolist()

    X_test.drop('tr_build_id', inplace=True, axis=1)
    X_test.drop('tr_status', inplace=True, axis=1)
    
    max_batch_size = 16
    min_batch_size = 1
    
    
    for alg in algorithm:
        
        
        cur_batch_size = max_batch_size
        if alg == 'BATCHSTOP4':
            if cur_batch_size < 4:
                continue



        Y_result = []
        grouped_batch = []
        actual_group_results = []
        group_duration = []
        num_feature = 4 
        length_of_test = len(Y_test)

        project_reqd_builds = []
        project_missed_builds = []
        project_build_duration = []
        project_saved_builds = []
        project_delays = []
        project_bad_builds = []
        project_batch_delays = []
        project_batch_medians = []
        project_ci = []

        commit = predictor.predict(X_test)
        for c in confidence:
            
            print('Processing {} at batch size {} for {} at confidence {}'.format(alg, cur_batch_size, p, c))
            
            ci = [Y_test[0]]
            batch_median = []
            batch_delays = 0

            pass_streak = Y_test[0]
            total_builds = 0
            missed_builds = 0
            miss_indexes = []
            build_indexes = []
            delay_durations = []

            if pass_streak == 0:
                saved_builds = 0
            else:
                saved_builds = 1

            index = 1

            while index < len(X_test):
                value = commit[index]
                #we're setting a confidence of 'c' builds on SBS, if more than 'c' passes have been suggested in a row, we don't want to trust sbs

                #if predict[0][1] > threshold:
                #    value = 1
                #else:
                #    value = 0
                #print('Build {} : predict_proba={}\tprediction={}'.format(index, predict, value))


                if pass_streak < c :
                    

                    if value == 0:
                        while True:

                            grouped_batch = list(X_test[index : index+cur_batch_size])
                            actual_group_results = list(Y_test[index : index+cur_batch_size])

                            if alg == 'BATCHBISECT':
                                if len(actual_group_results) != cur_batch_size:
                                    fb = 0
                                    while fb < len(actual_group_results):
                                        total_builds += 1
                                        ci.append(0)
                                        batch_delays += len(actual_group_results) - fb
                                        batch_median.append(cur_batch_size-fb-1)
                                        fb += 1
                                        index += 1
                                else:
                                    if len(miss_indexes) > 0:
                                        if miss_indexes[-1] < index:
                                            for l in range(len(miss_indexes)):
                                                e = miss_indexes.pop()
                                                delay_durations.append(index - e + 1)

                                    batch_total = batchbisect(actual_group_results)
                                    batch_delays += cur_batch_size*(cur_batch_size-1)/2
                                    ci.extend([0 for clb in range(cur_batch_size)])
                                    batch_median.extend([cur_batch_size-clb-1 for clb in range(cur_batch_size)])
                                    total_builds += batch_total

                            elif alg == 'BATCHSTOP4':
                                if len(actual_group_results) != cur_batch_size:
                                    fb = 0
                                    while fb < len(actual_group_results):
                                        total_builds += 1
                                        ci.append(0)
                                        batch_delays += len(actual_group_results) - fb
                                        batch_median.append(cur_batch_size-fb-1)
                                        fb += 1
                                        index += 1
                                else:
                                    if len(miss_indexes) > 0:
                                        if miss_indexes[-1] < index:
                                            for l in range(len(miss_indexes)):
                                                e = miss_indexes.pop()
                                                delay_durations.append(index - e + 1)


                                    batch_durations = 0

                                    batch_total = batchstop4(actual_group_results)

                                    batch_delays += cur_batch_size*(cur_batch_size-1)/2
                                    ci.extend([0 for clb in range(cur_batch_size)])
                                    batch_median.extend([cur_batch_size-clb-1 for clb in range(cur_batch_size)])
                                    total_builds += batch_total

                            elif alg == 'BATCHDIVIDE4':
                                if len(actual_group_results) != cur_batch_size:
                                    fb = 0
                                    while fb < len(actual_group_results):
                                        total_builds += 1
                                        ci.append(0)
                                        batch_delays += len(actual_group_results) - fb
                                        batch_median.append(cur_batch_size-fb-1)
                                        fb += 1
                                        index += 1
                                else:
                                    if len(miss_indexes) > 0:
                                        if miss_indexes[-1] < index:
                                            for l in range(len(miss_indexes)):
                                                e = miss_indexes.pop()
                                                delay_durations.append(index - e + 1)

                                    batch_total = batchdivide4(actual_group_results)
                                    batch_delays += cur_batch_size*(cur_batch_size-1)/2
                                    ci.extend([0 for clb in range(cur_batch_size)])
                                    batch_median.extend([cur_batch_size-clb-1 for clb in range(cur_batch_size)])
                                    total_builds += batch_total

                            if 0 in actual_group_results:
                                index += cur_batch_size
                                cur_batch_size = exp_update(0, actual_group_results, cur_batch_size, 3, min_size=get_minimum_for(alg))
                                grouped_batch.clear()
                                actual_group_results.clear()
                            else:
                                break
                        index += cur_batch_size
                        
                        if len(actual_group_results) > 0:
                            cur_batch_size = exp_update(1, actual_group_results, cur_batch_size, 3, min_size=get_minimum_for(alg))
                        pass_streak = 1
                        
                        grouped_batch.clear()
                        actual_group_results.clear()


                    else:
                        pass_streak += 1
                        ci.append(1)
                        saved_builds += 1
                        if Y_test[index] == 0:
                            missed_builds += 1
                            miss_indexes.append(index)

                        #seeing only one build
                        index += 1

                else:
                                        
                    while True:

                        grouped_batch = list(X_test[index : index+cur_batch_size])
                        actual_group_results = list(Y_test[index : index+cur_batch_size])


                        if alg == 'BATCHBISECT':
                            if len(actual_group_results) != cur_batch_size:
                                fb = 0
                                while fb < len(actual_group_results):
                                    total_builds += 1
                                    ci.append(0)
                                    batch_delays += len(actual_group_results) - fb
                                    batch_median.append(cur_batch_size-fb-1)
                                    fb += 1
                                    index += 1
                            else:

                                if len(miss_indexes) > 0:
                                    if miss_indexes[-1] < index:
                                        for l in range(len(miss_indexes)):
                                            e = miss_indexes.pop()
                                            delay_durations.append(index - e + 1)

                                batch_total = batchbisect(actual_group_results)

                                batch_delays += cur_batch_size*(cur_batch_size-1)/2
                                batch_median.extend([cur_batch_size-clb-1 for clb in range(cur_batch_size)])

                                ci.extend([0 for clb in range(cur_batch_size)])
                                total_builds += batch_total

                        elif alg == 'BATCHSTOP4':
                            if len(actual_group_results) != cur_batch_size:
                                fb = 0
                                while fb < len(actual_group_results):
                                    total_builds += 1
                                    ci.append(0)
                                    batch_delays += len(actual_group_results) - fb
                                    batch_median.append(cur_batch_size-fb-1)
                                    fb += 1
                                    index += 1
                            else:

                                if len(miss_indexes) > 0:
                                    if miss_indexes[-1] < index:
                                        for l in range(len(miss_indexes)):
                                            e = miss_indexes.pop()
                                            delay_durations.append(index - e + 1)

                                batch_total = batchstop4(actual_group_results)

                                batch_delays += cur_batch_size*(cur_batch_size-1)/2
                                batch_median.extend([cur_batch_size-clb-1 for clb in range(cur_batch_size)])
                                ci.extend([0 for clb in range(cur_batch_size)])
                                total_builds += batch_total
                        elif alg == 'BATCHDIVIDE4':
                            if len(actual_group_results) != cur_batch_size:
                                fb = 0
                                while fb < len(actual_group_results):
                                    total_builds += 1
                                    ci.append(0)
                                    batch_delays += len(actual_group_results) - fb
                                    batch_median.append(cur_batch_size-fb-1)
                                    fb += 1
                                    index += 1
                            else:

                                if len(miss_indexes) > 0:
                                    if miss_indexes[-1] < index:
                                        for l in range(len(miss_indexes)):
                                            e = miss_indexes.pop()
                                            delay_durations.append(index - e + 1)

                                batch_total = batchdivide4(actual_group_results)

                                batch_delays += cur_batch_size*(cur_batch_size-1)/2
                                batch_median.extend([cur_batch_size-clb-1 for clb in range(cur_batch_size)])
                                ci.extend([0 for clb in range(cur_batch_size)])
                                total_builds += batch_total
                        
                        
                        if 0 in actual_group_results:
                            index += cur_batch_size
                            cur_batch_size = exp_update(0, actual_group_results, cur_batch_size, 3, min_size=get_minimum_for(alg))
                            grouped_batch.clear()
                            actual_group_results.clear()

                        else:
                            break
                    index += cur_batch_size
                    if len(actual_group_results) > 0:
                        cur_batch_size = exp_update(1, actual_group_results, cur_batch_size, 3, min_size=get_minimum_for(alg))
                    pass_streak = 1
                    grouped_batch.clear()
                    actual_group_results.clear()

            mi = 0
            while len(miss_indexes) > 0:
                    m_index = miss_indexes.pop()
                    delay_durations.append(length_of_test - m_index + 1)

            project_reqd_builds.append(total_builds)
            project_missed_builds.append(missed_builds)
            project_saved_builds.append(saved_builds)
            project_delays.append(delay_durations)
            project_batch_delays.append(batch_delays)
            project_batch_medians.append(batch_median)
            project_ci.append(ci)

            if len(ci) != len(commit):
                print(len(ci))
                print(len(commit))
                print('PROBLEM!')
            else:
                #print('NO PROBLEM!')
                pass

        for i in range(len(confidence)):
            #print([p, alg, cur_batch_size, confidence[i], 100*project_reqd_builds[i]/length_of_test, 100*project_missed_builds[i]/length_of_test, project_build_duration[i], 100*project_saved_builds[i]/length_of_test, project_delays[i], length_of_test, project_batch_delays[i]])
            writer.writerow([ver, p, alg, cur_batch_size, confidence[i], 100*project_reqd_builds[i]/length_of_test, 100*project_missed_builds[i]/length_of_test, 100*project_saved_builds[i]/length_of_test, project_delays[i], length_of_test, project_batch_delays[i], project_batch_medians[i], project_ci[i]])
    result_file.close()

In [7]:
for pr in project_list:
    for i in range(0,10):
        static_rule(pr, i)

Processing BATCHBISECT at batch size 16 for junit-team-junit5/junit-team-junit5.csv at confidence 2
Processing BATCHBISECT at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 3
Processing BATCHBISECT at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 4
Processing BATCHBISECT at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 5
Processing BATCHBISECT at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 6
Processing BATCHBISECT at batch size 3 for junit-team-junit5/junit-team-junit5.csv at confidence 7
Processing BATCHBISECT at batch size 3 for junit-team-junit5/junit-team-junit5.csv at confidence 8
Processing BATCHBISECT at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 9
Processing BATCHBISECT at batch size 3 for junit-team-junit5/junit-team-junit5.csv at confidence 10
Processing BATCHBISECT at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 11
Process

Processing BATCHSTOP4 at batch size 16 for junit-team-junit5/junit-team-junit5.csv at confidence 2
Processing BATCHSTOP4 at batch size 16 for junit-team-junit5/junit-team-junit5.csv at confidence 3
Processing BATCHSTOP4 at batch size 16 for junit-team-junit5/junit-team-junit5.csv at confidence 4
Processing BATCHSTOP4 at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 5
Processing BATCHSTOP4 at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 6
Processing BATCHSTOP4 at batch size 12 for junit-team-junit5/junit-team-junit5.csv at confidence 7
Processing BATCHSTOP4 at batch size 12 for junit-team-junit5/junit-team-junit5.csv at confidence 8
Processing BATCHSTOP4 at batch size 12 for junit-team-junit5/junit-team-junit5.csv at confidence 9
Processing BATCHSTOP4 at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 10
Processing BATCHSTOP4 at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 11
Processing B

Processing BATCHDIVIDE4 at batch size 16 for junit-team-junit5/junit-team-junit5.csv at confidence 2
Processing BATCHDIVIDE4 at batch size 15 for junit-team-junit5/junit-team-junit5.csv at confidence 3
Processing BATCHDIVIDE4 at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 4
Processing BATCHDIVIDE4 at batch size 12 for junit-team-junit5/junit-team-junit5.csv at confidence 5
Processing BATCHDIVIDE4 at batch size 12 for junit-team-junit5/junit-team-junit5.csv at confidence 6
Processing BATCHDIVIDE4 at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 7
Processing BATCHDIVIDE4 at batch size 12 for junit-team-junit5/junit-team-junit5.csv at confidence 8
Processing BATCHDIVIDE4 at batch size 12 for junit-team-junit5/junit-team-junit5.csv at confidence 9
Processing BATCHDIVIDE4 at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 10
Processing BATCHDIVIDE4 at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confid

Processing BATCHBISECT at batch size 16 for junit-team-junit5/junit-team-junit5.csv at confidence 2
Processing BATCHBISECT at batch size 16 for junit-team-junit5/junit-team-junit5.csv at confidence 3
Processing BATCHBISECT at batch size 16 for junit-team-junit5/junit-team-junit5.csv at confidence 4
Processing BATCHBISECT at batch size 16 for junit-team-junit5/junit-team-junit5.csv at confidence 5
Processing BATCHBISECT at batch size 16 for junit-team-junit5/junit-team-junit5.csv at confidence 6
Processing BATCHBISECT at batch size 16 for junit-team-junit5/junit-team-junit5.csv at confidence 7
Processing BATCHBISECT at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 8
Processing BATCHBISECT at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 9
Processing BATCHBISECT at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 10
Processing BATCHBISECT at batch size 9 for junit-team-junit5/junit-team-junit5.csv at confidence 11
Pr

Processing BATCHBISECT at batch size 16 for ratpack-ratpack/ratpack-ratpack.csv at confidence 2
Processing BATCHBISECT at batch size 9 for ratpack-ratpack/ratpack-ratpack.csv at confidence 3
Processing BATCHBISECT at batch size 9 for ratpack-ratpack/ratpack-ratpack.csv at confidence 4
Processing BATCHBISECT at batch size 9 for ratpack-ratpack/ratpack-ratpack.csv at confidence 5
Processing BATCHBISECT at batch size 9 for ratpack-ratpack/ratpack-ratpack.csv at confidence 6
Processing BATCHBISECT at batch size 9 for ratpack-ratpack/ratpack-ratpack.csv at confidence 7
Processing BATCHBISECT at batch size 9 for ratpack-ratpack/ratpack-ratpack.csv at confidence 8
Processing BATCHBISECT at batch size 9 for ratpack-ratpack/ratpack-ratpack.csv at confidence 9
Processing BATCHBISECT at batch size 9 for ratpack-ratpack/ratpack-ratpack.csv at confidence 10
Processing BATCHBISECT at batch size 9 for ratpack-ratpack/ratpack-ratpack.csv at confidence 11
Processing BATCHBISECT at batch size 9 for ratp

...

Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 4
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 5
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 6
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 7
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 8
Processing BATCHSTOP4 at batch size 12 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 9
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 10
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 11
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 12
Processing BATCHSTOP4 at batch size 9 for 

Processing BATCHBISECT at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 19
Processing BATCHBISECT at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 20
Processing BATCHDIVIDE4 at batch size 16 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 2
Processing BATCHDIVIDE4 at batch size 12 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 3
Processing BATCHDIVIDE4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 4
Processing BATCHDIVIDE4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 5
Processing BATCHDIVIDE4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 6
Processing BATCHDIVIDE4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 7
Processing BATCHDIVIDE4 at batch size 12 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 8
Processing BATCHDIVIDE4 a

Processing BATCHSTOP4 at batch size 16 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 2
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 3
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 4
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 5
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 6
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 7
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 8
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 9
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 10
Processing BATCHSTOP4 at batch size 9 for ch

Processing BATCHDIVIDE4 at batch size 16 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 2
Processing BATCHDIVIDE4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 3
Processing BATCHDIVIDE4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 4
Processing BATCHDIVIDE4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 5
Processing BATCHDIVIDE4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 6
Processing BATCHDIVIDE4 at batch size 12 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 7
Processing BATCHDIVIDE4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 8
Processing BATCHDIVIDE4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 9
Processing BATCHDIVIDE4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 10
Processing BATCHDIVIDE4 a

Processing BATCHSTOP4 at batch size 16 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 2
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 3
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 4
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 5
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 6
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 7
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 8
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 9
Processing BATCHSTOP4 at batch size 9 for checkstyle-checkstyle/checkstyle-checkstyle.csv at confidence 10
Processing BATCHSTOP4 at batch size 9 for ch